# Example: Varying Sample Positions

Here outlines a basic sample set method where the sample position is incremented depending on the defined plate.
* Plate sample position logic defined (for ANSI-48Vial2mLHolder) 
* Samples in sample set defined in for loop, varying the sample position

# Import

In [1]:
from OptiHPLCHandler import EmpowerHandler

handler = EmpowerHandler(
    project="WebAPI_test", 
    address="XXXXXXXX", 
    allow_login_without_context_manager=False)

# Configuration of methods, nodes, systems and plates

Using get attributes to: obtain method from list_method, node from list_nodes, system from list_systems within defined node and define the plate type from list_plates.

In [2]:
with handler:
    list_methods = handler.GetMethodList()
    method = list_methods[-1]  # most recent method
    print(f"Method Set Name: {method}")

    list_nodes = handler.GetNodeNames()
    node = list_nodes[40]
    print(f"Node: {node}")

    list_systems = handler.GetSystemNames(node=node)
    system = list_systems[0]
    print(f"System Configuration Name: {system}")

    list_plates = handler.GetPlateTypeNames()[:6]
    plate = list_plates[5]
    print(f"Plate: {plate}")
    plates = {"1": plate, "2": plate}

C:\Users\slcb\Desktop\Programming\OptiHPLCHandler\src\OptiHPLCHandler\empower_api_core.py:172: UserWarning: The password will be sent in plain text.
  warnings.warn("The password will be sent in plain text.")


Method Set Name: TEST_Fus4
Node: Epdkhqr01045
System Configuration Name: 457_UPLC_10
Plate: ANSI-48Vial2mLHolder


Defining sample position counting logic and varying sample position in sample_list

In [3]:
# Varying vial position

sample_set_method_name = "test_samplesetmethod2"

platelist = [1, 2]
rowlist = [i + 1 for i in range(8)]  # rows defined numerically, counting from 1 to 8.
columnlist = [chr(i + 65) for i in range(6)]
# columns defined alphabetically. chr(65)=A, thus list comprehension counts from A onwards through the number of columns (6).
viallist = []

for plate in platelist:
    for column in columnlist:
        for row in rowlist:
            viallist.append(f"{plate}:{column},{row}")


sample_list = []
number_of_samples = 3


for i in range(number_of_samples):
    sample_list.append(
        {
            "Method": method,
            "SamplePos": viallist[i],
            "SampleName": f"Sample",
            "InjectionVolume": 3,
        },
    )

sample_list

[{'Method': 'TEST_Fus4',
  'SamplePos': '1:A,1',
  'SampleName': 'Sample',
  'InjectionVolume': 3},
 {'Method': 'TEST_Fus4',
  'SamplePos': '1:A,2',
  'SampleName': 'Sample',
  'InjectionVolume': 3},
 {'Method': 'TEST_Fus4',
  'SamplePos': '1:A,3',
  'SampleName': 'Sample',
  'InjectionVolume': 3}]

Post the experiment sample set method and run it

In [4]:
with handler:
    handler.PostExperiment(
        sample_set_method_name=sample_set_method_name,
        sample_list=sample_list,
        plates=plates,
        audit_trail_message=None,
    )
    handler.RunExperiment(
        sample_set_method=sample_set_method_name,
        sample_set_name=sample_set_method_name,
        system=system,
        node=node,
    )